In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier


In [ ]:
!pip install imblearn

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
data_raw = pd.read_csv('Fraud_check.csv')
data_raw

In [ ]:
data = data_raw.copy()
x = 0
for i in data_raw['Taxable.Income']:
    if i <= 30000:
        data['Taxable.Income'][x] = 'Risky'
    else:
        data['Taxable.Income'][x] = 'Good'
    x += 1
data

In [ ]:
sns.countplot(data['Taxable.Income'])

In [ ]:
y = data['Taxable.Income']
X = data.drop('Taxable.Income', axis = 1)
X['Undergrad'] = X['Undergrad'].map({'NO' : 0, 'YES' : 1})
X['Marital.Status'] = X['Marital.Status'].map({'Single' : 0, 'Married' : 1, 'Divorced' : 2})
X['Urban'] = X['Urban'].map({'NO' : 0, 'YES' : 1})
X

# Resampling the data

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
kfold = KFold(10)
accuracy = []
for i in range(1,201):
    forest = RandomForestClassifier(n_estimators = i)
    results = cross_val_score(forest, X, y, cv = kfold)
    accuracy.append(np.mean(results))
accuracy

In [ ]:
n_est_ideal = accuracy.index(max(accuracy[2:]))
n_est_ideal

In [ ]:
plt.figure(figsize = (15,15))
plt.plot(range(1,201), accuracy, drawstyle = 'steps-post')
plt.xlabel('Number of Trees', fontsize = 20)
plt.ylabel('Mean Accuracy', fontsize = 20)
plt.title('Mean Accuracy vs No. of trees', fontsize = 20)
plt.grid()

In [ ]:
resample = SMOTETomek(random_state = 42)
X_res, y_res = resample.fit_resample(X, y)
(X_res.shape, y_res.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, random_state = 42, test_size = 0.2)

In [ ]:
forest = RandomForestClassifier(random_state = 42, n_estimators = n_est_ideal)
forest.fit(X_train, y_train)

In [ ]:
predictions = forest.predict(X_test)
np.mean(predictions == y_test)

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
cf_mat = confusion_matrix(y_test, predictions)
cf_mat

In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
plot_confusion_matrix(forest, X_test, y_test, ax = ax)
ax.set_xlabel('Predicted Label', fontsize = 20)
ax.set_ylabel('True Label', fontsize = 20)
plt.show()